In [ ]:
%sql
 CREATE TABLE Billing (
    id BIGINT,
    billing_type STRING NOT NULL,
    invoice_date DATE NOT NULL,
    currency STRING NOT NULL
)
USING DELTA;

CREATE TABLE product_items (
    id BIGINT,
    billing_id BIGINT,
    product STRING,
    amount DECIMAL(10, 2),
    type STRING,
    valid_from DATE,
    valid_to DATE
)
USING DELTA;

CREATE TABLE Exchange (
    date DATE NOT NULL,
    from_currency STRING NOT NULL,
    to_currency STRING NOT NULL,
    exchange_rate DECIMAL(10, 5) NOT NULL
)
USING DELTA;

In [ ]:
%sql
INSERT INTO Billing (id, billing_type, invoice_date, currency) VALUES
(1, 'invoice', '2019-01-15', 'usd'),
(2, 'invoice', '2019-01-15', 'cad'),
(3, 'credit_note', '2019-03-15', 'cad'),
(4, 'invoice', '2019-03-15', 'eur'),
(5, 'invoice', '2019-04-01', 'eur'),
(6, 'credit_note', '2019-04-18', 'eur'),
(7, 'invoice', '2019-04-18', 'usd'),
(8, 'invoice', '2019-05-27', 'usd');

INSERT INTO product_items (id, billing_id, product, amount, type, valid_from, valid_to) VALUES
(1, 1, 'product1', 11500, 'recurring', '2019-01-15', '2019-12-31'),
(2, 1, 'product2', 5000, 'one_shot', NULL, NULL),
(3, 2, 'product3', 7893, 'recurring', '2019-01-15', '2020-01-14'),
(4, 2, 'product4', 16000, 'recurring', '2019-01-15', '2020-01-14'),
(5, 3, 'product3', 7893, 'recurring', '2019-01-15', '2020-01-14'),
(6, 3, 'product4', 16000, 'recurring', '2019-01-15', '2020-01-14'),
(7, 4, 'product1', 12350, 'recurring', '2019-03-15', '2019-12-31'),
(8, 5, 'product2', 5500, 'one_shot', NULL, NULL),
(9, 5, 'product3', 9000, 'recurring', '2019-04-01', '2020-03-31'),
(10, 5, 'product4', 14321, 'recurring', '2019-04-01', '2020-03-31'),
(11, 6, 'product2', 5500, 'one_shot', NULL, NULL),
(12, 6, 'product3', 9000, 'recurring', '2019-04-01', '2020-03-31'),
(13, 6, 'product4', 14321, 'recurring', '2019-04-01', '2020-03-31'),
(14, 7, 'product2', 4050, 'one_shot', NULL, NULL),
(15, 8, 'product6', 9000, 'one_shot', NULL, NULL);

INSERT INTO Exchange (date, from_currency, to_currency, exchange_rate) VALUES
('2019-01-15', 'cad', 'usd', 1.332),
('2019-01-15', 'eur', 'usd', 0.978),
('2019-01-15', 'usd', 'usd', 1),
('2019-03-15', 'cad', 'usd', 1.402),
('2019-03-15', 'eur', 'usd', 0.911),
('2019-03-15', 'usd', 'usd', 1),
('2019-04-01', 'cad', 'usd', 1.360),
('2019-04-01', 'eur', 'usd', 0.947),
('2019-04-01', 'usd', 'usd', 1),
('2019-04-18', 'cad', 'usd', 1.319),
('2019-04-18', 'eur', 'usd', 0.966),
('2019-04-18', 'usd', 'usd', 1),
('2019-05-27', 'cad', 'usd', 1.388),
('2019-05-27', 'eur', 'usd', 0.932),
('2019-05-27', 'usd', 'usd', 1);


## Annual Recurring Revenue (ARR) as of a specific date (2020-01-28)

from pyspark.sql.functions import col, datediff, round, sum as spark_sum

# Load the Delta tables
billing_df = spark.table("default.billing")
product_item_df = spark.table("default.product_items")
exchange_df = spark.table("default.exchange")

# Define the target date
target_date = '2020-01-28'

# Filter recurring items and join tables
recurring_items = (
    product_item_df
    .filter(col("type") == "recurring")  # Only recurring items
    .join(billing_df, product_item_df.billing_id == billing_df.id)  # Join with Billing
    .join(exchange_df, 
          (billing_df.currency == exchange_df.from_currency) & 
          (billing_df.invoice_date == exchange_df.date))  # Join with Exchange
    .filter((col("valid_from") <= target_date) & (col("valid_to") >= target_date))  # Valid on the target date
)

# Calculate days of validity and ARR in USD
arr_usd = (
    recurring_items
    .withColumn("days_validity", datediff(col("valid_to"), col("valid_from")))  # Calculate days of validity
    .withColumn("annualized_revenue", 
                (col("amount") / col("days_validity")) * 360 * col("exchange_rate"))  # Calculate ARR
    .filter(col("billing_type") == "invoice")  # Only include invoices
    .agg(round(spark_sum("annualized_revenue"), 2).alias("ARR_USD"))  # Sum up ARR and round to 2 decimal places
)

# Display the result
arr_usd.show()


## Annual Recurring Revenue (ARR) for the last day of each month between June and December 2019

from pyspark.sql.functions import col, datediff, round, sum as spark_sum, sequence, last_day, lit, explode

# Load the Delta tables
billing_df = spark.table("default.billing")  # Adjust database.table name if necessary
product_item_df = spark.table("default.product_items")  # Adjust database.table name
exchange_df = spark.table("default.exchange")  # Adjust database.table name

# Generate the sequence of dates (from June 2019 to December 2019)
date_sequence = spark.sql("SELECT SEQUENCE(DATE('2019-06-01'), DATE('2019-12-01'), INTERVAL 1 MONTH) AS month_start")

# Explode the sequence and get the last day of each month
date_range = (
    date_sequence
    .withColumn("month_start", explode(col("month_start")))  # Explode the sequence into individual rows
    .withColumn("arr_date", last_day(col("month_start")))  # Get the last day of each month
    .select("arr_date")
)

# Filter recurring items and join tables
recurring_items = (
    product_item_df
    .filter(col("type") == "recurring")  # Only recurring items
    .join(billing_df, product_item_df.billing_id == billing_df.id)  # Join with Billing
    .join(exchange_df, 
          (billing_df.currency == exchange_df.from_currency) & 
          (billing_df.invoice_date == exchange_df.date))  # Join with Exchange
)

# Calculate ARR for each month
monthly_arr = (
    recurring_items
    .crossJoin(date_range)  # Combine with all months
    .filter((col("valid_from") <= col("arr_date")) & (col("valid_to") >= col("arr_date")))  # Valid on arr_date
    .withColumn("days_validity", datediff(col("valid_to"), col("valid_from")))  # Calculate days of validity
    .withColumn("annualized_revenue", 
                (col("amount") / col("days_validity")) * 360 * col("exchange_rate"))  # Calculate ARR
    .filter(col("billing_type") == "invoice")  # Only include invoices
    .groupBy("arr_date")  # Group by month
    .agg(round(spark_sum("annualized_revenue"), 2).alias("ARR_USD"))  # Sum ARR for the month
    .orderBy("arr_date")
)

# Display the result
monthly_arr.show()